# Game theory Algorithms:

## cooperative games:

**Les jeux coopératifs:** sont des activités où les joueurs travaillent ensemble pour atteindre un objectif commun, plutôt que de s'affronter.  
Et voici algorithme Shaplay value utilisés dans la théorie des jeux coopératifs:

### Shaplay value algorithm:

Le Shapley Value est un algorithme utilisé en théorie des jeux pour attribuer de manière équitable la contribution de chaque joueur à la valeur totale d'un jeu coopératif. Il se base sur toutes les permutations possibles des joueurs pour calculer leur contribution marginale moyenne.

In [81]:
from itertools import chain, combinations

def calculate_shapley_value(player, players, value_function):
    n = len(players)
    shapley_value = 0

    for subset in powerset(players):
        if player not in subset:
            cardinality = len(subset)
            coalition_with_player = tuple(subset) + (player,)
            marginal_contribution = value_function(coalition_with_player) - value_function(subset)
            shapley_value += (factorial(cardinality) * factorial(n - cardinality - 1) / factorial(n)) * marginal_contribution

    return shapley_value

def powerset(iterable):
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s) + 1))

def factorial(num):
    return 1 if num == 0 else num * factorial(num - 1)

def split_prize(players, value_function, total_prize):
    shapley_values = {player: calculate_shapley_value(player, tuple(players), value_function) for player in players}
    total_shapley_values = sum(shapley_values.values())

    prize_distribution = {player: total_prize * shapley_value / total_shapley_values for player, shapley_value in shapley_values.items()}

    return prize_distribution

Voila une exemple dans l'aquelle on utilise shapley values pour split faily a prize 20000$,  
between tow players ('A', 13000) et ('B', 10500)

In [82]:
# Example usage
def value_function(coalition):
    return sum(player[1] for player in coalition)  # Sum of player values in the coalition

players = [("A", 13000), ("B", 10500)]
total_prize = 20000

prize_distribution = split_prize(players, value_function, total_prize)

print("Prize Distribution:", prize_distribution)


Prize Distribution: {('A', 13000): 11063.829787234043, ('B', 10500): 8936.170212765957}


**Shapley Values dans le problème des joueurs dans une équipe de football:** Le problème classique où les valeurs de Shapley sont utilisées est le problème de la coalition dans les jeux coopératifs.
Un exemple courant est le problème des joueurs dans une équipe de football qui contribuent de manière collective à la victoire de l'équipe.
Imaginons que nous ayons une équipe de football avec trois joueurs : Alice, Bob et Charlie.
Chacun d'eux contribue de manière unique au succès de l'équipe, et nous voulons attribuer une valeur à chaque joueur en fonction de sa contribution.
Voici un exemple de script Python pour calculer les valeurs de Shapley dans ce contexte :

In [83]:
from itertools import permutations

def coalition_value(coalition, marginal_contributions):
    # Calcule la valeur d'une coalition
    return sum(marginal_contributions[player] for player in coalition)

def shapley_values(players, value_function):
    # Calcule les valeurs de Shapley pour chaque joueur
    num_players = len(players)
    shapley_values = {player: 0.0 for player in players}

    for player in players:
        for perm in permutations(players):
            coalition = list(perm)
            player_index = coalition.index(player)
            without_player = coalition[:player_index] + coalition[player_index+1:]
            with_player = coalition[:player_index] + [player] + coalition[player_index:]

            shapley_values[player] += (value_function(with_player) - value_function(without_player)) / factorial(num_players)

    return shapley_values

# Exemple avec une équipe de football
players = ['Alice', 'Bob', 'Charlie']

# Fonction de valeur : la somme des forces des joueurs dans l'équipe
def team_value(coalition):
    return sum(player_strength[player] for player in coalition)

# Forces des joueurs
player_strength = {'Alice': 10, 'Bob': 8, 'Charlie': 6}

# Calcul des valeurs de Shapley
shapley = shapley_values(players, team_value)

# Affichage des résultats
for player, value in shapley.items():
    print(f"{player}: {value}")


Alice: 20.0
Bob: 15.999999999999998
Charlie: 12.0


## non-cooperative games:

### Mixed Strategy Nash Equilibrium:

Stratégie mixte en théorie des jeux signifie qu'un joueur prend des décisions de manière probabiliste,  
mélangeant différentes actions avec des probabilités spécifiques.  
L'équilibre de Nash en stratégies mixtes est une solution où chaque joueur rend ses adversaires indifférents   
entre différentes actions possibles, et aucun joueur n'a intérêt à dévier de sa stratégie mixte.

In [84]:
import random

def jouer_pierre_papier_ciseaux(strategie_joueur1, strategie_joueur2):
    actions = ["pierre", "papier", "ciseaux"]

    # Joueurs choisissent une action en fonction de leur stratégie mixte
    action_joueur1 = random.choices(actions, weights=strategie_joueur1)[0]
    action_joueur2 = random.choices(actions, weights=strategie_joueur2)[0]

    # Afficher les choix des joueurs
    print(f"Joueur 1 choisit {action_joueur1}")
    print(f"Joueur 2 choisit {action_joueur2}")

    # Déterminer le résultat du jeu
    if action_joueur1 == action_joueur2:
        return "Match nul"
    elif (
        (action_joueur1 == "pierre" and action_joueur2 == "ciseaux") or
        (action_joueur1 == "papier" and action_joueur2 == "pierre") or
        (action_joueur1 == "ciseaux" and action_joueur2 == "papier")
    ):
        return "Joueur 1 remporte la partie"
    else:
        return "Joueur 2 remporte la partie"

# Exemple d'utilisation avec des stratégies mixtes
strategie_joueur1 = [0.3, 0.4, 0.3]  # Probabilités pour pierre, papier, ciseaux
strategie_joueur2 = [0.4, 0.3, 0.3]  # Probabilités pour pierre, papier, ciseaux

resultat = jouer_pierre_papier_ciseaux(strategie_joueur1, strategie_joueur2)
print(f"Résultat : {resultat}")


Joueur 1 choisit ciseaux
Joueur 2 choisit pierre
Résultat : Joueur 2 remporte la partie


In [86]:
import numpy as np

def calculate_mixed_strategy(payoff_matrix_player1, payoff_matrix_player2):
    num_strategies_player1 = len(payoff_matrix_player1)
    num_strategies_player2 = len(payoff_matrix_player2[0])

    # Formulate the linear programming problem for player 1
    c1 = np.ones(num_strategies_player1) * -1
    A_eq1 = np.ones((1, num_strategies_player1))
    b_eq1 = np.array([1])

    # Formulate the linear programming problem for player 2
    c2 = np.ones(num_strategies_player2) * -1
    A_eq2 = np.ones((1, num_strategies_player2))
    b_eq2 = np.array([1])

    # Solve the linear programming problems
    result_player1 = np.linalg.lstsq(A_eq1, b_eq1, rcond=None)[0]
    result_player2 = np.linalg.lstsq(A_eq2, b_eq2, rcond=None)[0]

    return result_player1, result_player2

# Exemple d'utilisation avec le jeu de la "Bataille des sexes"
payoff_matrix_player1 = np.array([
    [3, 3],
    [2, 5],
    [0, 6]
])

payoff_matrix_player2 = np.array([
    [3, 2],
    [2, 6],
    [3, 1]
])

result_player1, result_player2 = calculate_mixed_strategy(payoff_matrix_player1, payoff_matrix_player2)

print("Stratégie Mixte du Joueur 1:", result_player1)
print("Stratégie Mixte du Joueur 2:", result_player2)


Stratégie Mixte du Joueur 1: [0.33333333 0.33333333 0.33333333]
Stratégie Mixte du Joueur 2: [0.5 0.5]
